In [1]:
import psycopg2

In [15]:
redshift_host='redshift-cluster-1.ckboajuk6yuq.us-east-1.redshift.amazonaws.com'
redshift_dbname='dev'
redshift_user='awsuser'
redshift_password='Adebukola81#'
redshift_port = '5439'

In [21]:
try:
    redshift_host='redshift-cluster-1.ckboajuk6yuq.us-east-1.redshift.amazonaws.com'
    redshift_dbname='dev'
    redshift_user='awsuser'
    redshift_password='Adebukola81#'
    redshift_port = '5439'
    # Establish the connection
    connection = psycopg2.connect(
        dbname=redshift_dbname,
        user=redshift_user,
        password=redshift_password,
        host=redshift_host,
        port=redshift_port
    )
    
    # Create a cursor object
    cursor = connection.cursor()
    
    # Execute a simple query
    cursor.execute("SELECT version();")
    
    # Fetch and print the result of the query
    version = cursor.fetchone()
    print(f"Connected to Redshift, PostgreSQL version: {version}")
    
    # Close the cursor and connection
    cursor.close()
    connection.close()
except Exception as e:
    print(f"Error connecting to Redshift: {e}")

Error connecting to Redshift: could not translate host name "redshift-cluster-1.ckboajuk6yuq.us-east-1.redshift.amazonaws.com" to address: No such host is known. 



In [23]:
import boto3
import time
import psycopg2

# Configuration
AWS_REGION = "us-east-1"
CLUSTER_IDENTIFIER = "redshift-cluster-1"
DB_NAME = "dev"
DB_USER = "awsuser"
DB_PASSWORD = "Adebukola81#"
CLUSTER_TYPE = "multi-node"
NODE_TYPE = "dc2.large"
NUMBER_OF_NODES = 1
ROLE_NAME = "myRedshiftRole"
#S3_BUCKET = "your-s3-bucket-name"
#S3_KEY = "path/to/your/datafile.txt"
#TABLE_NAME = "your_table_name"
#ACCOUNT_ID = "your-account-id"

def create_iam_role():
    iam = boto3.client('iam', region_name=AWS_REGION)
    try:
        role = iam.create_role(
            RoleName=ROLE_NAME,
            AssumeRolePolicyDocument="""{
                "Version": "2012-10-17",
                "Statement": [
                    {
                        "Effect": "Allow",
                        "Principal": {
                            "Service": "redshift.amazonaws.com"
                        },
                        "Action": "sts:AssumeRole"
                    }
                ]
            }"""
        )
        iam.attach_role_policy(
            RoleName=ROLE_NAME,
            PolicyArn="arn:aws:iam::aws:policy/AmazonS3ReadOnlyAccess"
        )
        return role['Role']['Arn']
    except iam.exceptions.EntityAlreadyExistsException:
        role = iam.get_role(RoleName=ROLE_NAME)
        return role['Role']['Arn']

def create_redshift_cluster(role_arn):
    redshift = boto3.client('redshift', region_name=AWS_REGION)
    redshift.create_cluster(
        ClusterType=CLUSTER_TYPE,
        NodeType=NODE_TYPE,
        NumberOfNodes=NUMBER_OF_NODES,
        DBName=DB_NAME,
        ClusterIdentifier=CLUSTER_IDENTIFIER,
        MasterUsername=DB_USER,
        MasterUserPassword=DB_PASSWORD,
        IamRoles=[role_arn]
    )
    print("Creating Redshift cluster...")
    while True:
        response = redshift.describe_clusters(ClusterIdentifier=CLUSTER_IDENTIFIER)
        status = response['Clusters'][0]['ClusterStatus']
        if status == 'available':
            cluster_endpoint = response['Clusters'][0]['Endpoint']['Address']
            print(f"Cluster available. Endpoint: {cluster_endpoint}")
            return cluster_endpoint
        print("Waiting for cluster to become available...")
        time.sleep(60)

def execute_etl(cluster_endpoint):
    conn_params = {
        "dbname": DB_NAME,
        "user": DB_USER,
        "password": DB_PASSWORD,
        "host": cluster_endpoint,
        "port": 5439
    }

    with psycopg2.connect(**conn_params) as connection:
        with connection.cursor() as cursor:
            # Create table
            cursor.execute(f"""
                CREATE TABLE IF NOT EXISTS {TABLE_NAME} (
                    id INT,
                    name VARCHAR(50),
                    age INT
                );
            """)
            # Copy data from S3
            cursor.execute(f"""
                COPY {TABLE_NAME}
                FROM 's3://{S3_BUCKET}/{S3_KEY}'
                IAM_ROLE 'arn:aws:iam::{ACCOUNT_ID}:role/{ROLE_NAME}'
                DELIMITER ','
                IGNOREHEADER 1;
            """)
            connection.commit()
    print("ETL completed.")

def delete_redshift_cluster():
    redshift = boto3.client('redshift', region_name=AWS_REGION)
    redshift.delete_cluster(
        ClusterIdentifier=CLUSTER_IDENTIFIER,
        SkipFinalClusterSnapshot=True
    )
    print("Deleting Redshift cluster...")
    while True:
        try:
            redshift.describe_clusters(ClusterIdentifier=CLUSTER_IDENTIFIER)
            print("Waiting for cluster deletion...")
            time.sleep(60)
        except redshift.exceptions.ClusterNotFoundFault:
            print("Cluster deleted.")
            break

In [25]:
create_iam_role()

NoCredentialsError: Unable to locate credentials

In [ ]:
if __name__ == "__main__":
    try:
        role_arn = create_iam_role()
        cluster_endpoint = create_redshift_cluster(role_arn)
        execute_etl(cluster_endpoint)
    finally:
        delete_redshift_cluster()